![alt text](images/uspas.png)

# Concepts of Accelerator Science and Technology 
# Computer Lab: Transverse Dynamics
##### Author: N. Neveu

This session deals with transverse dynamics in two sections. The first demonstrates the evolution of a beam envelope by tracking a particle through a thin lens FODO lattice. The second demonstrates how offsets in a lattice can impact beam trajectory in a linear machine. 

### Python Notes:

- Highlight (click into) a notebook cell and press Shift+Enter to execute, or use the play button at the top of the window.
- Make sure you execute notebook cells in order. You must re-execute prior cells if you change something needed in a cell above where you are working.
- You can also execute the whole notebook by using 'Run all cells' under the 'Run' tab.
- '2**2' represents two squared, i.e. 2^2
- A colon (:) means all values in that dimension. i.e. array[:,2] = all rows, second column
- You can change the plot limits by adjusting the numbers in 'plt.ylim()'
- Using the '@' symbol does matrix multiplication

----------

In [ ]:
### Run this cell to load needed packages
%matplotlib inline
from ipywidgets import interactive

# Importing plotting library
import matplotlib
from matplotlib import pyplot as plt

#Setting resolution of plot (changes size too)
matplotlib.rcParams['figure.dpi'] = 100

# Importing math library
import numpy as np

# 1. FODO Lattices 
FODO lattices are one of the most commonly used beam line configurations, and have been for many decades in accelerator physics. 

![alt text](images/fodo1.png)
![alt text](images/fodo2.png)


Consider a focusing and defocusing quad that are spaced 1 meter apart....

## **Q1.1) What does FODO stand for?** 

## **Q1.2) What types of magnets are used for F and D?**

## **Q1.3) In the next cell, define the transfer matrices for the three types of elements (matrices) in a FODO lattice.**
### Transverse matrices are typically 2x2 and represent 1 dimension (x or y). i.e. replace the zeros.

In [ ]:
def fodo_mats(L, f):
    drift = np.array([[0, 0],
                      [0, 0]])

    fquad  = np.array([[0, 0],
                       [0, 0]])

    dquad  = np.array([[0, 0],
                       [0, 0]])

    mats = {'drift': drift, 
            'fquad': fquad, 
            'dquad': dquad,}
    
    return mats

def matrix_calcs(lattice, x0):
    # lattice = fodo matrices 
    # x0 = initial conditions
    data = [x0,] 
    for i, ele in enumerate(lattice):
        data.append(ele @ data[i])  
    return data

## **Q1.4) In the next cell, we define the initial conditions of our particles. As shown in the homework, this can be represented by a 2x1 matrix. Explain what these offsets mean physically.** 

In [ ]:
xinit_offset = 0.5
xinit_angle  = 0.3
d1   = 1 # meters - drift length
qkl  = 2  
    
mats = fodo_mats(d1, qkl)

In [ ]:
def plot_fodo(xinit_offset, xinit_angle):
    # Plotting the data    
    plt.figure(figsize=[10,3])
    x0 = np.array([[xinit_offset],
                   [xinit_angle]])
    lattice = [mats['drift'], mats['fquad'], mats['drift'], mats['dquad'], mats['drift']]
    data = matrix_calcs(lattice, x0) 
    xvalues = [i[0] for i in data]
    zvalues = [0, 1, 1, 2, 2,3]
    plt.plot(zvalues, xvalues, markersize=10)
    plt.ylabel('x [m]', size=14)
    plt.xlabel('z [m]', size=14)
    plt.grid()    
    plt.show()

In [ ]:
# You can adjust the angle and offset ranges here:
xmin  = -0.1
xmax  = 0.1
xstep = 0.02
amin  = -1.0
amax  = 1.5
astep = 0.2

interactive_plot = interactive(plot_fodo, xinit_offset=(xmin, xmax, xstep), xinit_angle=(amin, amax, astep))
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot

## **Q1.5) Play with the initial conditions to get an understanding of the particle trajectory.**
A) What initial conditions result in a straight line? Why?   
B) Are there conditions that will result in $x(z=0) = x(z=3)$? i.e. the same x offset at the start and end of the trajectory.
C) Choose a trajectory and explain the path the particle is taking. Paste a screen shot here, you can also annotate.   

## **Q1.6) Next we will plot multiple particle trajectories on the same plot.** 
Again you can adjust the plot by changing the initial condition ranges.   
Explain what is being shown here? 

In [ ]:
def plot_fodo_all(xinit_offset, xinit_angle):
    # Plotting the data    
    x0 = np.array([[xinit_offset],
                   [xinit_angle]])
    lattice = [mats['drift'], mats['fquad'], mats['drift'], mats['dquad'], mats['drift']]
    data    = matrix_calcs(lattice, x0) 
    xvalues = [i[0] for i in data]
    zvalues = [0, 1, 1, 2, 2,3]
    plt.plot(zvalues, xvalues, markersize=10,label='')

In [ ]:
plt.figure(figsize=[10,3])
plt.ylabel('x [m]', size=14)
plt.xlabel('z [m]', size=14)
plt.grid()  

# plot multiple initial conditions
for xoff in np.arange(xmin,xmax, xstep):
    for xang in np.arange(amin,amax, astep):
        plot_fodo_all(xoff,xang)

## **OPTIONAL: Plot the changes in x' (Q5). What do you observe about the particle behavior?**

# 2. LCLS-SC Example

The Linac Coherent Light Source [(LCLS)](https://lcls.slac.stanford.edu/) at SLAC is an Free Electron Laser [(FEL)](https://uspas.fnal.gov/materials/materials-table.shtml) facility that provides x-rays to a variety of users. The x-rays are generated with two operating linear accelerators. The copper accelerator (LCLS-CU) has been operating since 2009, and provides electrons up to 15 GeV. The superconding accelerator (LCLS-SC), started commissioning in 2022, and will be able to provide electrons of 8 GeV, after the high energy (HE) upgrade in 2026. 

![alt text](images/slac_layout.png)

In LCLS-SC, niobium cavities are cooled to 2K during operation. Each cryomodule has 8 cavities that provide ~100-120 MeV of energy to the beam. Currently, we have 35 cryomodules that provide a total beam energy of about 4 GeV. 

![alt text](images/cryomodule.jpeg)

Each cryomodule also has a quadrupole and correccting magnets behind the last cavity. This effectively means we do not have beam orbit information in the middle of the cryomodule (~10 meters long). If the beam (or dark current) enters the cryomdule at an angle, or off-axis, the beam may get a kick from the cryomodule magnets when it exits the cryomodule. This exact scenario has happened at LCLS-SC in , and we used basic matrix calculations to estimate where dark current would hit the next module. 

Consider the beam line layout in the next figure. The magnets in cryomodule 34 are located at z = 635 m. Higher than normal radiation levels were observed between cavities 6 and 7 on cryomodule 34. The start of cryomodule 35 is at 636.5 m. We need to caculate if it is possible for off phase dark current (from cryomodule 35) to hit the wall of cryomodule 35 between cavities 6 and 7. 

 ![alt text](images/cm_matrix_layout.png)

Let's simplify this problem to match a standard matrix calculation for a quad and corrector. Given the distances in the figure above, we can re-write the problem as: 

![alt text](images/simple_layout.png)

Where $q_{cm}$ is the location of the quad AND corrector magnets, and we flipped the distances so that the exit of cryomodule 35 is at $x_0$. The drift distances are indicated by $d_1$ and $d_2$. Given this configuration, assume the magnet strength of the quad is $kx=0.0019$ KG-m and $ky=0.0025$ KG-m. The effective length of the quad is $l_q = 0.1$ m. Note: we are ignoring any focusing effects from the accelerating fields in the cavities. This is not completely realistic, but rather an estimate. We also can interchange $x$ and $y$ dimension by using the magnet strengths and offsets in those respective dimensions. In the next few questions we will step through determining if beam will hit the side of the cryomodule.

Now, let's define a function that will do the matrix calculation to find the beam offset at $x_2$. 

$$
\begin{pmatrix}
  x_2 \\
  {x'}_2  
\end{pmatrix} = M_d M_m M_d
\begin{pmatrix}
  x_0 \\
  {x'}_0 
\end{pmatrix} 
$$

Where $M_t$ is the transfer function that represents the drift and magent matrices.TODO:fix x prime notation.

In [ ]:
# The @ sign in Python does matrix multiplication
def calculate_x2(x0, drift1, drift2, magnet):
    return drift2 @ magnet @ drift1 @ x0

## **Q2.1) Fill in the constants we will need to solve for the beam offset at $x_2$.**

In [ ]:
# Constants
d1 = 0.0 # m, drift 1
d2 = 0.0 # m, drift 2
# Multiply field strength by length to get T units
ql = 0.0 # m, effective length of the quad
klx = 0.0*ql  # T-m = 19  G-m, quad strength in x 
kly = 0.0*ql # T-m = -25 G-m, quad strength in y - taking absolute value
energy = 0.6 # GeV, beam energy after traveling through 4 cavities

## **Q2.2) Fill in an initial x and y position offset and trajectory angle. Assume something in the millimeter and millirad range. How does this compare to a typcial beam pipe radius?**

In [ ]:
# Initial conditions
# 10 degrees = ~0.17 rad
xinit_offset = 0.001 # m, Say we start w/ 1 mm offset
xinit_angle  = 0.01 # rad, Say we start with a very small angle (best case)

yinit_offset = 0.001 # m, Say we start w/ 1 mm offset
yinit_angle  = 0.0 # rad, Say we start with no angle (best case)

## **Q2.3) Based on the constants and values we defined, fill in the matrix definitions that we need to describe the beam line. Some of the zeros below are placeholders.**

In [ ]:
x0 = np.array([[0], # matrix for initial x0 offset
               [0]])

y0 = np.array([[0], # matrix for initial y0 offset
               [0]])

m_d1 = np.array([[0, 0], # matrix for d1
                 [0, 0]])

m_d2 = np.array([[0, 0], # matrix for d2
                 [0, 0]])

q_x = np.array([[0, 0], # matrix for quad, x dimension
                [0, 0]])

q_y = np.array([[0, 0], # matrix for quad, y dimension
                [0, 0]])

## **Q2.4) Call the calculate_x2 function, with the variables we defined above for cryomodule 34 and 35. Print the resulting offsets in x and y. Based on these results, will we hit the wall?**

In [ ]:
x2 = calculate_x2(x0, m_d1, m_d2, q_x)
y2 = calculate_x2(y0, m_d1, m_d2, q_y)
print('Beam offset at CM34 in x: ', x2[0][0]*1000, 'mm')
print('Beam offset at CM34 in y: ', y2[0][0]*1000, 'mm')

# 3 Challenge problem (optional)

The quadrupole magnets in each cryomodule are combination magnets that include corrector magnets for the x and y dimenison as well. You can approximate these magnets as small and weak dipoles. 
Dark current passing through the correctors may also be kicked into the wall. Calculate how big of an initial offset would cause the beam to deflect 10 mm at x2. Note corrector strengths are currently at: $x_{34} = 0.019$ KG-m and $y_{34} = -0.025$ KG-m. 
Representing the correctors as small dipoles, the field strength (B), magnet length (L), bend angle ($\theta$), arc length ($\rho$), and radius can be related. 

## **Q3.1) Let's define a function to calculate $\rho$ and $\theta$ for a dipole magnet. How does the beam energy impact the radius?**

In [ ]:
# Define function related to constant values we need to model a dipole matrix
# Assuming momentum and kinetic energy differences are neglible (electrons)
def calc_rho_theta(energy, k, l):
    rho   = energy / (0.3*k*l)
    theta = l/rho
    print('rho', rho, 'theta', theta)
    return rho, theta

## **Q3.2) Define the relevant constants for the corrector magnets.**

In [ ]:
cl = 0.0 # m, corrector length in meters
kx = 0.0  # T-m = 19  G-m, corrector strength in x 
ky = 0.0 # T-m = -25 G-m, corrector strength in y - taking absolute value

## **Q3.3) Define the x and y matrices for a simplified dipole/corrector magnet.**

In [ ]:
rhox, thetax = calc_rho_theta(energy, kx, cl) 
rhoy, thetay = calc_rho_theta(energy, ky, cl)

c_cmx = np.array([[0, 0],
                 [0, 0]])

c_cmy = np.array([[0, 0],
                  [0, 0]])

In [ ]:
cxinit_offset = 0.001
cyinit_offset = 0.001

cxinit_angle = 0.08
cyinit_angle = 0.08

x0c = np.array([[0], # matrix for initial x0 offset
               [0]])

y0c = np.array([[0], # matrix for initial y0 offset
               [0]])

xc_offsets = calc_x2(x0c, m_d1, m_d2, c_cmx)
yc_offsets = calc_x2(y0c, m_d1, m_d2, c_cmy) 

print('Beam offset at CM34 in x: ', xc_offsets[0][0]*1000, 'mm')
print('Beam offset at CM34 in y: ', yc_offsets[0][0]*1000, 'mm')